In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
f = open('accessfiles.py', 'wb')
f.write(streaming_body_1.read())

import accessfiles

In [1]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8 MB 8.5 kB/s  eta 0:00:01    |███████████▌                    | 77.9 MB 10.3 MB/s eta 0:00:140:07��█               | 115.2 MB 17.1 MB/s eta 0:00:07MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 50.4 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=e916ddf1bf768f8df820fe63475307f74e969108b1eab6b8845e3d045d96645a
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [2]:
from zipfile import ZipFile
import time
import numpy as np
from pandas import DataFrame
from operator import add

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

from accessfiles import load_files

load_files()

In [37]:

from pyspark.sql import Row

movies=sc.textFile("movies.dat")
id_rdd=movies.map(lambda row:Row(MovieID=row.split("::")[0]))
m_id = spark.createDataFrame(id_rdd)


## Extract the title
title_rdd=movies.map(lambda row:row.split("::")[1])
name_rdd=title_rdd.map(lambda mv:Row(Title=mv.split("(")[0]))
name = spark.createDataFrame(name_rdd)

## Extract the year
year_rdd = title_rdd.map(lambda mv:Row(Year=mv.split("(")[-1].split(')')[0]))
year = spark.createDataFrame(year_rdd)


## Extract the genre
genre_rdd=movies.map(lambda row:Row(Genres=row.split("::")[2]))
genre = spark.createDataFrame(genre_rdd)


## For appending the dataframes, we need to import monotonically_increasing_id
from pyspark.sql import functions as F

add1=m_id.withColumn("id", F.monotonically_increasing_id()).join(name.withColumn("id", F.monotonically_increasing_id()), on=["id"]).drop("id")


add2=year.withColumn("id", F.monotonically_increasing_id()).join(genre.withColumn("id",F.monotonically_increasing_id()),on=["id"]).drop("id")

m_result=add1.withColumn("id", F.monotonically_increasing_id()).join(add2.withColumn("id", F.monotonically_increasing_id()), on=["id"]).drop("id")

## This will give us the valid data with schema
m_result.show()


+--------+--------------------+----+--------------------+
|MoviesID|               Title|Year|              Genres|
+--------+--------------------+----+--------------------+
|    1011| Herbie Rides Again |1974|Adventure|Childre...|
|     744|Brothers in Trouble |1995|               Drama|
|    3459|             Gothic |1986|        Drama|Horror|
|    3244|  Goodbye Girl, The |1977|      Comedy|Romance|
|     195|Something to Talk...|1995|Comedy|Drama|Romance|
|    3676|         Eraserhead |1977|        Drama|Horror|
|    3054|Pok�mon: The Firs...|1998|Animation|Children's|
|    2355|      Bug's Life, A |1998|Animation|Childre...|
|    1006|       Chamber, The |1996|               Drama|
|    3841|        Air America |1990|       Action|Comedy|
|     295|Pyromaniac's Love...|1995|      Comedy|Romance|
|     922|       Sunset Blvd. |1950|           Film-Noir|
|     813|   Larger Than Life |1996|              Comedy|
|    2294|               Antz |1998|Animation|Children's|
|    1795|Call

In [42]:
'''                                                                 
## Examples
m_result.where("MovieID=1").show()
m_result.filter("Genres == 'Action'").show()
m_result.select("Title").show()

'''

+--------------------+
|               Title|
+--------------------+
| Herbie Rides Again |
|Brothers in Trouble |
|             Gothic |
|  Goodbye Girl, The |
|Something to Talk...|
|         Eraserhead |
|Pok�mon: The Firs...|
|      Bug's Life, A |
|       Chamber, The |
|        Air America |
|Pyromaniac's Love...|
|       Sunset Blvd. |
|   Larger Than Life |
|               Antz |
|Callej�n de los m...|
|       Manxman, The |
|  Perez Family, The |
|Swiss Family Robi...|
|League of Their O...|
|Arsenic and Old L...|
+--------------------+
only showing top 20 rows



'                                                                 \n## Examples\nm_result.where("MovieID=1").show()\nm_result.filter("Genres == \'Action\'").show()\nm_result.select("Title").show()\n\n'

In [45]:
from accessfiles import create_download_link
m_result_pd = m_result.toPandas()
create_download_link(m_result_pd, filename='movies_df.csv')
